# Borrower Reliability Research


## Let's open the table and examine the general information about the data

In [1]:
import pandas as pd

In [ ]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Data Preprocessing

### Removing missing values

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

There are missing values in two columns. One of them is days_employed. You will handle the missing values in this column in the next step. The other column with missing values is total_income, which contains data about income. The income amount is most strongly influenced by the type of employment, so the missing values in this column should be filled with the median value for each employment type from the income_type column. For example, for a person with the employment type employee, the missing value in the total_income column should be filled with the median income among all records with the same employment type.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Handling outlier values

There may be artifacts (outliers) in the data—values that do not reflect reality and were caused by some error. One such artifact could be a negative number of days worked in the days_employed column. This is not normal for real data. Handle the values in this column by replacing all negative values with positive ones using the abs() method.

In [6]:
data['days_employed'] = data['days_employed'].abs()

For each employment type, display the median value of the work experience days_employed in days.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

For two types (unemployed and retirees), the values will be abnormally large. It is difficult to correct such values, so leave them as they are.

Let's display the list of unique values in the children column.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

There are two anomalous values in the children column. Remove the rows containing these anomalous values from the data dataframe.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Print the list of unique values from the children column again to ensure that the anomalies have been removed.

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Fill the missing values in the days_employed column with the median values for each income_type.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Make sure that all missing values are filled. Check yourself and display the number of missing values for each column again using two methods.

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Data Type Conversion

Replace the float data type in the total_income column with an integer using the astype() method.

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Duplicate Handling

Handle implicit duplicates in the education column. There are the same values written in different ways: using uppercase and lowercase letters. Convert them to lowercase.

In [14]:
data['education'] = data['education'].str.lower()

Display the number of duplicate rows in the data. If such rows are present, remove them.

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Data Categorization

Based on the ranges below, create a total_income_category column in the data dataframe with categories:

- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 and above — 'A'.

For example, a borrower with an income of 25000 should be assigned the category 'E', while a client earning 235000 should be assigned category 'B'. Use your own function named categorize_income() and the apply() method.

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Display the list of unique loan purposes from the purpose column.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Create a function that, based on the data in the purpose column, will create a new column purpose_category with the following categories:

- 'car operations',
- 'real estate operations',
- 'wedding planning',
- 'education'.

For example, if the purpose column contains the substring 'for buying a car', then the purpose_category column should contain 'car operations'.

Use your own function named categorize_purpose() and the apply() method. Review the data in the purpose column and determine which substrings will help you correctly assign the category.

In [20]:
def categorize_purpose(row):
    try:
        if 'car' in row:
            return 'car operations'
        elif 'residential' in row or 'real estate' in row:
            return 'real estate operations'
        elif 'wedding' in row:
            return 'wedding planning'
        elif 'education' in row:
            return 'education'
    except:
        return 'no category'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Exploratory Data Analysis

Let's find the relationship between the number of children and loan repayment on time. To begin, let's create a new function that generates pivot tables using the pivot_table() method. We will later use this function for other factors in the subsequent chapters of the research.

In [22]:
def create_pivot(data, index_col, value_col): 
    # Declare a function that creates pivot tables. It takes as arguments a dataframe (data),
    # a column by which we group the data (index_col), and the values for which we want to see the pivot table (value_col)
    
    pivot = data.pivot_table(index=index_col, values=value_col, aggfunc=['count', 'sum', 'mean']) 
    # Use the pivot_table() method, passing the arguments mentioned above, and adding the aggfunc function 
    # where we pass count for all loan applicants, sum for calculating defaulters, and mean for calculating the default rate

    pivot.columns = ['Total loan applicants', 'Total defaulters', 'Default rate'] 
    # Rename the columns with the resulting data correctly

    return pivot 
    # Return the result of the function

In [23]:
pivot_children = create_pivot(data, 'children', 'debt') 
# Pass the arguments to the function and calculate the values in the pivot table
pivot_children

,Всего кредитополучателей,Всего должников,Доля должников
children,,,
0,14091,1063,0.075438
1,4808,444,0.092346
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


Let's consider the largest categories — 0, 1, and 2 children. The default rate among these categories is distributed as follows:

- Among borrowers with no children - 7.5%
- Among borrowers with one child - 9.2%
- Among borrowers with two children - 9.5%

A certain pattern can be observed here, based on which we can form the following hypothesis: as the number of children increases, the default rate also increases. Based on this hypothesis, we can assert that a potential borrower with no children is more likely to repay the loan.

We will apply the exact same algorithm as with the number of children in the family. This time, let's look at the relationship status of the client. We will categorize clients by their relationship status and, within these categories, find the number and percentage of clients who had debt related to loan repayment.

In [24]:
pivot_family = create_pivot(data, 'family_status', 'debt') # Pass the function arguments and calculate the values in the pivot table.
pivot_family

,Всего кредитополучателей,Всего должников,Доля должников
family_status,,,
Не женат / не замужем,2796,273,0.097639
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246
гражданский брак,4134,385,0.093130
женат / замужем,12261,927,0.075606


The observed difference in percentages is quite small, but based on the results, an interesting hypothesis can be made: clients who are or were not married tend to fulfill their credit obligations worse than those who were married.

We can observe this in the categories "Not married" and "Cohabitation," where the debt percentage is 9.8% and 9.3%, respectively. After these, the categories "Divorced" (7%) and "Married" (7.6%) follow. The least amount of debt is found in the "Widowed" category, at 6.6%. However, it’s important to note that the percentage difference is minimal, so perhaps this factor (relationship status) should not be considered a decisive one.

For the third time, we will apply the same principle and try to find a pattern in loan repayment on time based on the income level of the bank's clients.

In [25]:
pivot_income = create_pivot(data, 'total_income_category', 'debt') # We will pass the function arguments and calculate the values in the pivot table.
pivot_income

,Всего кредитополучателей,Всего должников,Доля должников
total_income_category,,,
A,25,2,0.080000
B,5014,354,0.070602
C,15921,1353,0.084982
D,349,21,0.060172
E,22,2,0.090909


As with children, let's look at the largest categories presented - B and C. The distribution of debtors between these categories is as follows:

- Among borrowers with an income ranging from 200,000 to 1 million (B) - 7.1%
- Among borrowers with an income ranging from 50,000 to 200,000 (C) - 8.5%

Based on the results, we can hypothesize that as a potential client's income increases, their reliability improves. In other words, clients with higher income are more willing to repay their loans.

Let's consider the factor of loan purpose.

In [26]:
pivot_purpose = create_pivot(data, 'purpose_category', 'debt') # We will pass the arguments to the function and calculate the values in the pivot table.
pivot_purpose

,Всего кредитополучателей,Всего должников,Доля должников
purpose_category,,,
операции с автомобилем,4279,400,0.093480
операции с недвижимостью,10751,780,0.072551
получение образования,3988,369,0.092528
проведение свадьбы,2313,183,0.079118


Once again, we observe a small difference in percentages between the categories. The leaders in terms of debt are clients who took loans for "vehicle operations" and "education" purposes, with 9.4% and 9.3%, respectively. Clients who took loans for "real estate operations" and "wedding planning" have lower debt percentages, with 7.3% and 8%, respectively.

Based on the conclusions drawn from the results of the tasks above, missing values are found in the columns days_employed and total_income. Let's take a closer look at the possible reasons for their occurrence. These possible reasons can be divided into two main groups: issues during data collection and issues during data processing and provision.

In the first case, for example, these fields for data entry could have been optional, and the client simply decided not to provide this type of information. Additionally, in some cases, bank employees may have made errors while filling out or entering data, which led to missing values in the corresponding columns. This reason can also be attributed to human error.

In the second case, issues may have arisen during data processing or format conversion, leading to missing values in the columns. This reason can be referred to as a technical issue.

In our case, we used the median value for the columns total_income and days_employed. In both cases, by doing this, we did not lose other important data that we would have lost if we had removed rows with missing values from the dataframe. We preserved the central tendency of the data distribution across client employment types and avoided distortion that would have occurred if we had used the mean value.

The mean is the average value of a set of numbers. The median is the numerical value that separates the larger half of a data set from the smaller half, meaning it shows the central value in the sample. For example, we have test scores of students: 10, 15, 20, 25, and 100. The mean is 34, but this is an unreliable average since most of the scores are below 34. To assess students' performance, it may be more useful to use the median, which is 20, as it is closer to the majority of the scores. The mean is very sensitive to large outliers. An example of using the mean value would be measuring a patient's pulse: at different times, the pulse was 72, 74, 80, 76, and 78 beats per minute. The data doesn't vary much, so by calculating the arithmetic mean, we can accurately describe the patient's average pulse rate, which is 76 beats per minute.

### Conclusion

The aim of the research was to identify dependencies of loan default based on various client factors/characteristics—how do the factors/characteristics proposed by the client affect the potential borrower's ability to repay the bank loan?

As introductory information, we received a dataframe containing a range of data describing the clients. Before proceeding directly to data analysis and answering the client's questions, it was necessary to prepare the data for further work. During the data preprocessing, the following steps were carried out:

- The pandas library was imported, and data was read from the provided dataframe.
- The basic information about the dataframe was studied and checked: number of rows and columns, data types, and missing values.
- Missing values in the columns 'total_income' and 'days_employed' were processed.
- For filling missing values in the 'total_income' column, a for loop and logical indexing were used. The missing values were filled with the median income value for each income type.
- Missing values in the 'days_employed' column were also handled. The same method was used as for 'total_income', after removing anomalously large values using logical indexing. Negative values were corrected using the abs() function.
- Using the .sum() method, we confirmed that there were no more missing values in the dataframe.
- The data type in the 'total_income' column was changed from float to int using the .astype() method for easier number handling.
- We then proceeded to work with explicit duplicates, which were identified using the .duplicated().sum() method, and removed them with the .drop_duplicates() method.
- To facilitate analysis, we categorized the data in the 'purpose' and 'total_income' columns for more convenient analysis.

In the next stage of the project, we proceeded directly with the data analysis and answers to the following questions:

- Is there a dependency between the number of children and timely loan repayment?
- Is there a dependency between marital status and timely loan repayment?
- Is there a dependency between income level and timely loan repayment?
- How do different loan purposes affect timely repayment?

Before starting the analysis, we created a new function that generates pivot tables using the pivot_table() method. After analyzing the results, we came to the following conclusions:

- With an increase in the number of children, the debt for timely loan repayment increases. Based on this hypothesis, it can be stated that a potential borrower with no children is more likely to repay the loan.
- Clients who are not and have never been married perform worse in terms of loan repayment than those who are married.
- With an increase in the income of a potential client, their reliability increases. In other words, clients with higher income are more likely to repay loans.
- The leaders in defaults are clients who took loans for "car operations" and "education"—9.4% and 9.3% respectively. Clients who took loans for "real estate operations" and "wedding arrangements" have less default, with 7.3% and 8% respectively.
- A reliable client can be described as follows: the client is married, has no children, has an income above 200,000, and takes a loan for real estate operations or wedding arrangements.

An unreliable client can be described as follows: the client is not married, has 1 or more children, has an income up to 200,000, and takes a loan for car operations or education.

Based on the hypotheses presented, we recommend that the client more carefully collect and process client data and create a credit scoring system that, based on the collected data and the hypotheses proposed above, ranks clients by reliability.